In [83]:
import random
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import deque
%matplotlib inline

In [176]:
global verbose
verbose = True

In [4]:
def read_cards_file():
    with open("cards.txt") as f:
        try:
            cards = f.read().split('\n')[0:-1]
        except:
            for value in range(2, 11):
                for suit in ['c', 's', 'd', 'h']:
                    f.write(f'{value}{suit}\n')
            for value in ['J', 'Q', 'K', 'A']:
                for suit in ['c', 's', 'd', 'h']:
                    f.write(f'{value}{suit}\n')
            cards = f.read().split('\n')
            cards = cards[0:-1]
    return cards

In [5]:
cards_list = read_cards_file()

In [464]:
class Card:

# Had fields
#   value
#   name
#   suit
#   pip
    
    def __init__(self, pip=None, suit=None, name=None):
        self.turns_held = 0
        if pip:
            self.pip = pip
            self.suit = suit
            self.name = f'{pip}{suit}'
        elif name:
            self.name = name
            self.pip = name[0:-1]
            self.suit = name[-1:]
            
        try:
            self.value = int(self.pip)
        except:
            if self.pip=='J':
                self.value = 11
            elif self.pip=='Q':
                self.value = 12
            elif self.pip=='K':
                self.value = 13
            elif self.pip=='A':
                self.value = 1

In [465]:
def get_value_dic(*args, hand=None):
    
# hand argument assumes list of cards
    
    card_dic = {}
    if hand:
        for card in hand:
            if card.value in card_dic:
                card_dic[card.value] +=1
            else:
                card_dic[card.value] = 1
    else:
        for card in args:
            if card.value in card_dic:
                card_dic[card.value] +=1
            else:
                card_dic[card.value] = 1
    
    return card_dic

In [466]:
def get_keys_with_max_value(cards):
    
    if type(cards) == list:
        cards=get_value_dic(hand=cards)
    
    ret_keys = []
    if type(cards) == dict:
        if list(cards.values()).count(cards[max(cards, key=cards.get)]) == 1:
            ret_keys.append(max(cards, key=cards.get))
        else:
            for key in cards.keys():
                if cards[key] == cards[max(cards, key=cards.get)]:
                    ret_keys.append(key)
    return ret_keys

In [467]:
def get_key_with_min_value(cards):
    
    if type(cards) == list:
        cards=get_value_dic(hand=cards)
    
    ret_keys = []
    if type(cards) == dict:
        if list(cards.values()).count(cards[min(cards, key=cards.get)]) == 1:
            ret_keys.append(min(cards, key=cards.get))
        else:
            for key in cards.keys():
                if cards[key] ==  cards[min(cards, key=cards.get)]:
                    ret_keys.append(key)
    return ret_keys

In [468]:
class Deck(object):
    
# Has fields
#   cards
    
    def __init__(self, empty=False):
        self.cards = deque([])
        if not empty:
            for x in cards_list:
                self.cards.append(Card(name=x))
    
    def __iter__(self):
        return iter(self.cards)
    
    def __getitem__(self, item):
        return list(self.cards)[item]
    
    def __len__(self):
        return len(self.cards)
    
    def shuffle(self):
        prev_order = self.cards
        self.cards = deque([])
        for i in range(52):
            card_index = random.randint(-1, 51-i)
            self.cards.append(prev_order[card_index])
            del prev_order[card_index]
        return self
    
    def empty(self):
        self.cards = []
    
    def get_top_card(self, remove=True):
        if remove:
            return(self.cards.popleft())
        else:
            return self.cards[0]
    
    def add(self, new_card):
        self.cards.append(new_card)
    

In [469]:
class Player:

# Has fields:
#   hand
    
    def __init__(self, dealt_cards):
        if type(dealt_cards) == list:
            self.hand = dealt_cards
        elif type(dealt_cards) == Card:
            self.hand = []
            self.hand.append(dealt_cards)
        if verbose:
            print(f'{self.hand}')
    
    def take_card(self, card):
        self.hand.append(card)
        if verbose:
            print(f'appended {card.name}')
    
    def discard_card(self, card):
        self.hand.remove(card)
        if verbose:
            print(f'removed {card.name}')
        return card
    
    def show_hand(self):
        ret_str = ''
        for card in self.hand:
            ret_str += f'{card.name} '
        return ret_str[0:len(ret_str)-1]
    

In [470]:
class SpoonsPlayer(Player):
    
    def make_move(self, drawn_card):          
        self.take_card(drawn_card)
        uncommon_cards = get_key_with_min_value(self.hand)
        if verbose:
            print(f'uncommon cards = {uncommon_cards}')
        if len(uncommon_cards)==1:
            for card in self.hand:
                if card.value == uncommon_cards[0]:
                    return self.discard_card(card)
            print('SNGH make_move one uncommon card')
        else:          #This could be used for old card bias
            chosen_index = random.randint(0, len(uncommon_cards)-1)
            chosen_uncommon = uncommon_cards[chosen_index]
            if verbose:
                print(f'chosen index for discarded card is {chosen_index}')
                print(f'chosen discard card is {chosen_uncommon}')
            for card in self.hand:
                if card.value == chosen_uncommon:
                    return self.discard_card(card)
            print('SNGH make_move multiple uncommon card')

    def has_won(self):
        if [x.value for x in self.hand].count(self.hand[0].value) == 4:
            return True
        else:
            return False
    

In [471]:
class Game:
    
# Has fields:
#   deck
#   players
#   game_type
#   finished
#   discard_pile
    
    def __init__(self, number_of_players, game_type, true_deal=False):
        self.game_type = game_type
        self.finished = False
        self.deck = Deck().shuffle()
        self.discard_pile = Deck(empty=True)
        self.players = []
        if game_type == 'spoons':
            if true_deal:
                # true deal probably wont make a difference, but i honestly dont know
                pass
            else:
                for i in range(number_of_players):
                    self.players.append(SpoonsPlayer(list(self.deck)[4*i:4*(i+1)]))
    
    def run(self):
        while True:
            for player in self.players:
                if len(self.deck) == 0:
                    self.deck=self.discard_pile
                    self.discard_pile.empty()
                if self.players.index(player) == 0:
                    active_card = player.make_move(self.deck.get_top_card())
                elif self.players.index(player) != len(self.players)-1:
                    active_card = player.make_move(active_card)
                elif self.players.index(player) == len(self.players)-1:
                    self.discard_pile.add(player.make_move(self.deck.get_top_card()))
                else:
                    print('SNGH Game.run player index not matching expected case')
                if player.has_won():
                    return self.players.index(player)
                
            

In [472]:
class Simulation:

# Has fields:
#
    
    def __init__(self, number_of_players, number_of_games, game_type):
        

SyntaxError: unexpected EOF while parsing (<ipython-input-472-0d0651c40a51>, line 7)

In [473]:
game = Game(4, 'spoons')

In [474]:
game.run()

2